# Assignment 2: Building a Simple Index

In this assignment, we will build a simple search index, which we will use later for Boolean retrieval. The assignment tasks are again at the bottom of this document.

## Loading the Data

In [54]:
Summaries_file = 'data/cognition_Summaries.pkl.bz2'
Abstracts_file = 'data/cognition_Abstracts.pkl.bz2'

In [55]:
import pickle, bz2
from collections import namedtuple

Summaries = pickle.load( bz2.BZ2File( Summaries_file, 'rb' ) )

paper = namedtuple( 'paper', ['title', 'authors', 'year', 'doi'] )

for (id, paper_info) in Summaries.items():
    
    Summaries[id] = paper( *paper_info )
    
Abstracts = pickle.load( bz2.BZ2File( Abstracts_file, 'rb' ) )

Let's have a look at what the data looks like for an example of a paper:

In [56]:
Summaries[28102890]

paper(title='Handgrip Strength in Old and Very Old Adults: Mood, Cognition, Function, and Mortality.', authors=['Stessman J', 'Rottenberg Y', 'Fischer M', 'Hammerman-Rozenberg A', 'Jacobs JM'], year=2017, doi='10.1111/jgs.14509')

In [57]:
Abstracts[28102890]

'To determine the trajectory of handgrip strength (HGS) from age 70 to 90 and its association with mood, cognition, functional status, and mortality.'

## Some Utility Functions

We'll define some utility functions that allow us to tokenize a string into terms, perform linguistic preprocessing on a list of terms, as well as a function to display information about a paper in a nice way. Note that these tokenization and preprocessing functions are rather naive. We will improve them in a later assignment.

In [58]:
def tokenize(text):
    """
    Function that tokenizes a string in a rather naive way. Can be extended later.
    """
    return text.split(' ')

def preprocess(tokens):
    """
    Perform linguistic preprocessing on a list of tokens. Can be extended later.
    """
    result = []
    for token in tokens:
        result.append(token.lower())
    return result

print(preprocess(tokenize("Lorem ipsum dolor sit AMET")))

['lorem', 'ipsum', 'dolor', 'sit', 'amet']


In [59]:
from IPython.display import display, HTML
import re

def display_summary( id, show_abstract=False, show_id=True, extra_text='' ):
    """
    Function for printing a paper's summary through IPython's Rich Display System.
    Trims long author lists, and adds a link to the paper's DOI (when available).
    """
    s = Summaries[id]
    lines = []
    title = s.title
    if s.doi != '':
        title = '<a href=http://dx.doi.org/{:s}>{:s}</a>'.format(s.doi, title)
    title = '<strong>' + title + '</strong>'
    lines.append(title)
    authors = ', '.join( s.authors[:20] ) + ('' if len(s.authors) <= 20 else ', ...')
    lines.append(str(s.year) + '. ' + authors)
    if (show_abstract):
        lines.append('<small><strong>Abstract:</strong> <em>{:s}</em></small>'.format(Abstracts[id]))
    if (show_id):
        lines.append('[ID: {:d}]'.format(id))
    if (extra_text != ''):
         lines.append(extra_text)
    display( HTML('<br>'.join(lines)) )

display_summary(28102890)
display_summary(28102890, show_abstract=True)

## Creating our first index

We will now create an _inverted index_ based on the words in the titles and abstracts of the papers in our dataset. We will implement our inverted index as a Python dictionary with term strings as keys and posting lists (implemented as Python lists) as values. We include all the tokens we can find in the title and (if available) in the abstract:

In [60]:
from collections import defaultdict

inverted_index = defaultdict(list)

# This can take a few seconds:
for id in sorted(Summaries.keys()):
    term_set = set(preprocess(tokenize(Summaries[id].title)))
    if id in Abstracts:
        term_set.update(preprocess(tokenize(Abstracts[id])))
    for term in term_set:
        inverted_index[term].append(id)

Let's see what's in the index for the example term 'amsterdam':

In [61]:
print(inverted_index['amsterdam'])

[10547144, 17888546, 18289505, 19004529, 20594617, 20969529, 21482441, 22049191, 23232034, 23802509, 23964225, 25114679, 25598195, 25631781, 25684214, 25877267, 26976043, 27148107, 28462387, 28671504, 29029762, 29209183, 29406610, 29843832, 29848469, 31026198, 31082289, 31547145, 31730165, 33008394, 34203041, 34408904, 34541288, 34925184, 35149261]


We can now use this inverted index to answer simple one-word queries, for example to show all papers that contain the word 'utrecht':

In [62]:
query_word = 'utrecht'
for i in inverted_index[query_word]:
    display_summary(i, show_abstract=True)

----------

# Tasks

**Your name:** Szymon Dominik Modrzynski 2782521

### Task 1

Implement the function `and_merge` outlined below. This function takes two posting lists from the index that can be assumed to be sorted already, and it should return the result of the merging of the two lists with AND. The resulting list should therefore include all the elements that appear in both lists. As explained on the slides, this operation should take advantage of the input lists being sorted already, should not perform any additional sorting operation, and should go through each of the input lists just once. Then, test your function with an example.

In [63]:
list1 = [4, 5, 6, 8, 9]
list2 = [3, 4, 5, 6, 9, 10, 11]

def and_merge(pl1, pl2):
    merged_lists = []
    i = 0
    j = 0
    len1, len2 = len(pl1), len(pl2)
    while i<len1 and j<len2:
        if (pl1[i]==pl2[j]):
            merged_lists.append(pl1[i])
            i += 1
            j += 1
        elif (pl1[i]<pl2[j]): i+=1
        else: j+=1
    return merged_lists

print(and_merge(list1,list2))
print(and_merge(list2,list1))
print(and_merge(list1,list1))
print(and_merge(list2,list2))

[4, 5, 6, 9]
[4, 5, 6, 9]
[4, 5, 6, 8, 9]
[3, 4, 5, 6, 9, 10, 11]


### Task 2

Similarly as above, implement the function `or_merge` outlined below that executes an OR merging of the lists. The resulting list should therefore include all the elements that appear in at least one of the lists (note this is an 'inclusive OR' and not a 'XOR'). Again, this operation should take advantage of the input lists being sorted already, should not perform any additional sorting operation, and should go through each of the input lists just once. Elements that appear in both input list should only appear once in the output list. Test your function again with an example.

In [64]:
# list1 = [4, 5, 6, 8, 9]
# list2 = [3, 5, 6, 9, 10, 11]

def or_merge(pl1, pl2):
    merged_lists = []
    i = 0
    j = 0
    len1, len2 = len(pl1), len(pl2)

    while i<len1 and j<len2:
        if (pl1[i]<pl2[j]):
            merged_lists.append(pl1[i])
            i += 1
        elif (pl1[i]>pl2[j]):
            merged_lists.append(pl2[j])
            j += 1
        else:
            merged_lists.append(pl1[i])
            i += 1
            j += 1
            
    if (len1 > len2):
        for i in range(i, len1):
            merged_lists.append(pl1[i])
    else:
        for i in range(j, len2):
            merged_lists.append(pl2[i])

    return merged_lists


print(or_merge(list1,list2))
print(or_merge(list2,list1))
print(or_merge(list1,list1))
print(or_merge(list2,list2))


[3, 4, 5, 6, 8, 9, 10, 11]
[3, 4, 5, 6, 8, 9, 10, 11]
[4, 5, 6, 8, 9]
[3, 4, 5, 6, 9, 10, 11]


### Task 3

Construct a function called `and_query` that takes as input a single string, consisting of one or more words, and returns as function value a list of matching documents. `and_query`, as its name suggests, should require that all query terms are present in the documents of the result list.

For that, access the variable `inverted_index` from above and use the method `and_merge` that you defined. Also use the `tokenize` and `preprocess` functions we defined above to tokenize and preprocess your query.

Again demonstrate the working of your function with an example (choose one that leads to fewer than 100 hits to not overblow this notebook file).

In [65]:
def and_query(phrase: str):

    phrase = preprocess(tokenize(phrase))
    
    temp_index = []
    and_index = []

    for word in phrase:
        temp_index.append(inverted_index[word])

    # print("Len of temp_index: " + str(len(temp_index)))
    # print("Index[0]: " + str(len(temp_index[0])))
    # print("Index[1]: " + str(len(temp_index[1])))

    and_index = temp_index[0]
    for i in range(1, len(temp_index)):
        and_index = and_merge(temp_index[i], and_index)
        
    print("Number of papers: " + str(len(and_index)))

    return and_index

and_query('corpus thalamus')
# display_summary(11937069)
# display_summary(21091272)

Number of papers: 14


[11937069,
 21091272,
 21111408,
 22120144,
 24454909,
 25959627,
 25982658,
 26015269,
 26920674,
 28424397,
 29601896,
 34882280,
 34917014,
 35219873]

### Task 4

Construct another function called `or_query` that works in the same way as `and_query` you just implemented, but returns as function value the documents that contain _at least one_ of the words in the query, using the `or_merge` function you defined.

Demonstrate the working of this function also with an example (again, choose one that leads to fewer than 100 hits).

In [66]:
def or_query(phrase: str):

    phrase = preprocess(tokenize(phrase))
    
    temp_index = []
    or_index = []

    for word in phrase:
        temp_index.append(inverted_index[word])

    # print("Len of temp_index: " + str(len(temp_index)))
    # print("Index[0]: " + str(len(temp_index[0])))
    # print("Index[1]: " + str(len(temp_index[1])))

    or_index = temp_index[0]

    for i in range(1, len(temp_index)):
        or_index = or_merge(temp_index[i], or_index)
    
    print("Number of papers: " + str(len(or_index)))

    return or_index

or_query('6-OHDA SB-214111 Contusive')

Number of papers: 18


[11937069,
 15817263,
 18663473,
 20147561,
 21091272,
 21624440,
 22895713,
 23748382,
 24324416,
 24613235,
 26823963,
 28662892,
 29329678,
 30521934,
 33631218,
 33894299,
 35653829,
 35979400]

### Task 5

Why does `and_query('adult handgrip strength')` not return our example paper 28102890, even though it mentions handgrip strength and adults in the abstract? (You do not have to implement anything to fix this yet!)

Our normalization does not cover bringing the word to its root or alternating it any
way other than simply converting it into lower case, which does not allow for result
of search to be 'adult' when actual word in the paper is 'adults'

In [67]:
print(and_query('adult handgrip strength'))

Number of papers: 2
[33661082, 34439604]


# Submission

Submit the answers to the assignment via Canvas as a modified version of this Notebook file (file with `.ipynb` extension) that includes your code and your answers.

Before submitting, restart the kernel and re-run the complete code (**Kernel > Restart & Run All**), and then check whether your assignment code still works as expected.

Don't forget to add your name, and remember that the assignments have to be done **individually**, and that code sharing or copying are **strictly forbidden** and will be punished.